<a href="https://colab.research.google.com/github/wcl20/PyTorch-LeNet/blob/master/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch LeNet for MNIST

## 1. Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set random seed to make results reproducible
torch.manual_seed(0)
if torch.cuda.is_available():
  torch.cuda.manual_seed(0)
# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {torch.cuda.is_available()}")

Using GPU: True


## 2. Load Data

### 2.1 Data Preprocessing

In [0]:
# Define preprocess transformations
transform = transforms.Compose(
    [
     transforms.ToTensor(), 
     transforms.Normalize((0.1307,), (0.3081,))
    ]
)

### 2.2 Fetch Data

In [3]:
data_dir = "data/"
# Load training data
train_data = datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
# Load testing data
test_data = datasets.MNIST(root=data_dir, train=False, transform=transform)
# Get shape of data
image, _ = train_data[0]
print(f"Shape of image: {image.shape}")

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Shape of image: torch.Size([1, 28, 28])


### 2.3 Create DataLoaders

In [0]:
# Create DataLoader for training data
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
# Create DataLoader for testing data
test_loader = DataLoader(test_data, batch_size=1024, shuffle=False)

## 3. Define Model

In [5]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.pool = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    output = F.relu(self.conv1(x))            # Output size: N x 6 x 24 x 24
    output = self.pool(output)                # Output size: N x 6 x 12 x 12
    output = F.relu(self.conv2(output))       # Output size: N x 16 x 8 x 8
    output = self.pool(output)                # Output size: N x 16 x 4 x 4
    output = output.view(output.shape[0], -1) # Output size: N x 256
    output = F.relu(self.fc1(output))         # Output size: N x 120
    output = F.relu(self.fc2(output))         # Output size: N x 84
    output = self.fc3(output)                 # Output size: N x 10
    return output

model = LeNet().to(device)
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## 4. Train Model

### 4.1 Define Hyperparameters

In [0]:
# Learning rate
lr = 0.01
# Epochs
epochs = 20

### 4.1 Define Optimizer

In [0]:
optimizer = optim.SGD(model.parameters(), lr=lr)

### 4.2 Define Loss Function

In [0]:
criterion = nn.CrossEntropyLoss()

### 4.2 Model Training

In [9]:
model.train()
for epoch in range(epochs):
  for batch_idx, (inputs, targets) in enumerate(train_loader):
    # Use GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Zero gradient used by optimizers (prevent accumulating across batches)
    optimizer.zero_grad()
    # Predict output using model
    outputs = model(inputs)
    # Compute loss
    loss = criterion(outputs, targets)
    # Train model
    loss.backward()
    optimizer.step()
    # Print training status
    if batch_idx == 0:
      print(f"Train Epoch: {epoch} -- Loss: {loss.item()}")

Train Epoch: 0 -- Loss: 2.305492877960205
Train Epoch: 1 -- Loss: 0.8968749046325684
Train Epoch: 2 -- Loss: 0.27409130334854126
Train Epoch: 3 -- Loss: 0.15258769690990448
Train Epoch: 4 -- Loss: 0.10894595086574554
Train Epoch: 5 -- Loss: 0.1763058602809906
Train Epoch: 6 -- Loss: 0.0683472752571106
Train Epoch: 7 -- Loss: 0.12472997605800629
Train Epoch: 8 -- Loss: 0.06115805730223656
Train Epoch: 9 -- Loss: 0.17996187508106232
Train Epoch: 10 -- Loss: 0.043609410524368286
Train Epoch: 11 -- Loss: 0.06052558496594429
Train Epoch: 12 -- Loss: 0.01343480497598648
Train Epoch: 13 -- Loss: 0.05695465952157974
Train Epoch: 14 -- Loss: 0.07673761248588562
Train Epoch: 15 -- Loss: 0.06984882801771164
Train Epoch: 16 -- Loss: 0.0680498406291008
Train Epoch: 17 -- Loss: 0.10701178759336472
Train Epoch: 18 -- Loss: 0.04139029607176781
Train Epoch: 19 -- Loss: 0.13579612970352173


## 5. Test Model

In [10]:
model.eval()
correct = 0
with torch.no_grad():
  for inputs, targets in test_loader:
    # Use GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Predict output using model
    outputs = model(inputs)
    # Get Predicted class
    predictions = outputs.argmax(dim=1, keepdim=True)
    # Get number of correct predictions
    correct += predictions.eq(targets.view_as(predictions)).sum().item()
print(f"Accuracy Rate: {correct / len(test_loader.dataset)}")

Accuracy Rate: 0.9848


## 6. Save Model

In [0]:
torch.save(model.state_dict(), "model.pt")